In [3]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=6384015115a5e39acad293533af2bbb78deb0002327aee9ae48545a0062eb881
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [4]:
import chess
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("chess_games.csv")
pd.set_option('display.max_colwidth', None)
df["AN"][0]

'1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e4 6. Ne5 cxd5 7. Qa4+ Bd7 8. Nxd7 Nxd7 9. Nc3 Nf6 10. Qb3 Be7 11. Nxd5 Qa5+ 12. Nc3 O-O 13. Be2 b5 14. O-O Rad8 15. Bd2 Qc7 16. Rac1 Qd6 17. Qc2 Qe6 18. Nb1 Bd6 19. a3 Nb6 20. Qc6 Nfd5 21. Ba5 Rc8 22. Qb7 Qh6 23. h3 Nc4 24. Bxc4 bxc4 25. Qxd5 Rfd8 26. Qxe4 Rd7 27. Bc3 Re7 28. Qf3 Re6 29. Nd2 Rf6 30. Qg4 Re8 31. Ne4 Rg6 32. Qd7 Rf8 33. Nxd6 Rxd6 34. Qc7 Rg6 35. Qh2 Re8 36. d5 f6 37. d6 Rd8 38. Rfd1 1-0'

In [151]:
import chess
import numpy as np

class State:
    def __init__(self, board=None):
        self.board = board if board else chess.Board()

    def serialize(self):
        bstate = np.zeros(64, dtype=np.uint8)
        piece_map = {
            "P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6,
            "p": 9, "n": 10, "b": 11, "r": 12, "q": 13, "k": 14
        }

        for i in range(64):
            piece = self.board.piece_at(i)
            if piece:
                bstate[i] = piece_map[piece.symbol()]

        if self.board.has_queenside_castling_rights(chess.WHITE):
            bstate[0] = 7
        if self.board.has_kingside_castling_rights(chess.WHITE):
            bstate[7] = 7
        if self.board.has_queenside_castling_rights(chess.BLACK):
            bstate[56] = 15
        if self.board.has_kingside_castling_rights(chess.BLACK):
            bstate[63] = 15

        if self.board.ep_square is not None:
            bstate[self.board.ep_square] = 8

        bstate = bstate.reshape(8, 8)
        state = np.zeros((5, 8, 8), dtype=np.uint8)
        state[0] = (bstate >> 3) & 1
        state[1] = (bstate >> 2) & 1
        state[2] = (bstate >> 1) & 1
        state[3] = bstate & 1
        state[4] = np.full((8, 8), int(self.board.turn))
        return state

    def print_serialized_board(self):
        serialized = self.serialize()
        print("Serialized Board (5x8x8):")
        print(serialized)

# Test with initial board position
print("Initial board position:")
s = State()
print(s.board)
s.print_serialized_board()

# Test with a specific position
print("\nSpecific board position:")
fen = "r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4"
s = State(chess.Board(fen))
print(s.board)
s.print_serialized_board()

# Test after making a move
print("\nAfter making a move (e4):")
s = State()
s.board.push_san("e4")
print(s.board)
s.print_serialized_board()

Initial board position:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Serialized Board (5x8x8):
[[[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]]

 [[1 0 0 1 1 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 0 0 1 1 0 0 1]]

 [[1 1 1 0 1 1 1 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 0 1 1 1 1]]

 [[1 0 1 1 0 1 0 1]
  [1 1 1 1 1 1 1 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [1 0 1 1 0 1 0 1]]

 [[1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]]]

Specific bo

In [150]:
import chess

# Piece values
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

# Piece-square tables
PAWN_TABLE = [
    0,  0,  0,  0,  0,  0,  0,  0,
    50, 50, 50, 50, 50, 50, 50, 50,
    10, 10, 20, 30, 30, 20, 10, 10,
    5,  5, 10, 25, 25, 10,  5,  5,
    0,  0,  0, 20, 20,  0,  0,  0,
    5, -5,-10,  0,  0,-10, -5,  5,
    5, 10, 10,-20,-20, 10, 10,  5,
    0,  0,  0,  0,  0,  0,  0,  0
]

KNIGHT_TABLE = [
    -50,-40,-30,-30,-30,-30,-40,-50,
    -40,-20,  0,  0,  0,  0,-20,-40,
    -30,  0, 10, 15, 15, 10,  0,-30,
    -30,  5, 15, 20, 20, 15,  5,-30,
    -30,  0, 15, 20, 20, 15,  0,-30,
    -30,  5, 10, 15, 15, 10,  5,-30,
    -40,-20,  0,  5,  5,  0,-20,-40,
    -50,-40,-30,-30,-30,-30,-40,-50
]

BISHOP_TABLE = [
    -20,-10,-10,-10,-10,-10,-10,-20,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -10,  0,  5, 10, 10,  5,  0,-10,
    -10,  5,  5, 10, 10,  5,  5,-10,
    -10,  0, 10, 10, 10, 10,  0,-10,
    -10, 10, 10, 10, 10, 10, 10,-10,
    -10,  5,  0,  0,  0,  0,  5,-10,
    -20,-10,-10,-10,-10,-10,-10,-20
]

ROOK_TABLE = [
    0,  0,  0,  0,  0,  0,  0,  0,
    5, 10, 10, 10, 10, 10, 10,  5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    0,  0,  0,  5,  5,  0,  0,  0
]

QUEEN_TABLE = [
    -20,-10,-10, -5, -5,-10,-10,-20,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -10,  0,  5,  5,  5,  5,  0,-10,
    -5,  0,  5,  5,  5,  5,  0, -5,
    0,  0,  5,  5,  5,  5,  0, -5,
    -10,  5,  5,  5,  5,  5,  0,-10,
    -10,  0,  5,  0,  0,  0,  0,-10,
    -20,-10,-10, -5, -5,-10,-10,-20
]

KING_MIDDLE_GAME_TABLE = [
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -20,-30,-30,-40,-40,-30,-30,-20,
    -10,-20,-20,-20,-20,-20,-20,-10,
    20, 20,  0,  0,  0,  0, 20, 20,
    20, 30, 10,  0,  0, 10, 30, 20
]

KING_END_GAME_TABLE = [
    -50,-40,-30,-20,-20,-30,-40,-50,
    -30,-20,-10,  0,  0,-10,-20,-30,
    -30,-10, 20, 30, 30, 20,-10,-30,
    -30,-10, 30, 40, 40, 30,-10,-30,
    -30,-10, 30, 40, 40, 30,-10,-30,
    -30,-10, 20, 30, 30, 20,-10,-30,
    -30,-30,  0,  0,  0,  0,-30,-30,
    -50,-30,-30,-30,-30,-30,-30,-50
]

def evaluate_position(board):
    if board.is_checkmate():
        return -10000 if board.turn == chess.WHITE else 10000
    if board.is_stalemate() or board.is_insufficient_material():
        return 0

    score = 0

    # Determine game phase (middle game or endgame)
    endgame = is_endgame(board)

    # Material count and piece positioning
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            value = PIECE_VALUES[piece.piece_type]
            if piece.color == chess.WHITE:
                score += value
                score += get_piece_table_score(piece.piece_type, square, endgame)
            else:
                score -= value
                score -= get_piece_table_score(piece.piece_type, chess.square_mirror(square), endgame)

    # Pawn structure (doubled pawns penalty)
    for file in range(8):
        white_pawns = len(board.pieces(chess.PAWN, chess.WHITE) & chess.BB_FILES[file])
        black_pawns = len(board.pieces(chess.PAWN, chess.BLACK) & chess.BB_FILES[file])
        if white_pawns > 1:
            score -= 10 * (white_pawns - 1)
        if black_pawns > 1:
            score += 10 * (black_pawns - 1)

    # King safety (simplified)
    white_king_square = board.king(chess.WHITE)
    black_king_square = board.king(chess.BLACK)
    if white_king_square:
        if board.piece_at(chess.G1) == chess.Piece(chess.KING, chess.WHITE):
            score += 50  # Bonus for castled king
    if black_king_square:
        if board.piece_at(chess.G8) == chess.Piece(chess.KING, chess.BLACK):
            score -= 50  # Bonus for castled king

    return score

def get_piece_table_score(piece_type, square, endgame):
    if piece_type == chess.PAWN:
        return PAWN_TABLE[square]
    elif piece_type == chess.KNIGHT:
        return KNIGHT_TABLE[square]
    elif piece_type == chess.BISHOP:
        return BISHOP_TABLE[square]
    elif piece_type == chess.ROOK:
        return ROOK_TABLE[square]
    elif piece_type == chess.QUEEN:
        return QUEEN_TABLE[square]
    elif piece_type == chess.KING:
        return KING_END_GAME_TABLE[square] if endgame else KING_MIDDLE_GAME_TABLE[square]
    return 0

def is_endgame(board):
    # Simple endgame detection: if both sides have no queens or
    # if every side which has a queen has additionally no other pieces or one minorpiece maximum.
    white_queens = len(board.pieces(chess.QUEEN, chess.WHITE))
    black_queens = len(board.pieces(chess.QUEEN, chess.BLACK))
    white_pieces = len(board.pieces(chess.KNIGHT, chess.WHITE)) + len(board.pieces(chess.BISHOP, chess.WHITE)) + len(board.pieces(chess.ROOK, chess.WHITE))
    black_pieces = len(board.pieces(chess.KNIGHT, chess.BLACK)) + len(board.pieces(chess.BISHOP, chess.BLACK)) + len(board.pieces(chess.ROOK, chess.BLACK))

    return (white_queens == 0 and black_queens == 0) or \
           (white_queens == 1 and white_pieces <= 1 and black_queens == 0) or \
           (black_queens == 1 and black_pieces <= 1 and white_queens == 0)


#best_move = min(legal_moves, key=lambda move: evaluate_position(board1.push(move).copy()))  # Ensure we evaluate after copying the board
def minimax(board, depth, maximizing_player, alpha=float('-inf'), beta=float('inf')):
    """
    Minimax algorithm with alpha-beta pruning.

    :param board: Current board state
    :param depth: Depth of the search
    :param maximizing_player: True if it's white's turn (maximizing), False if it's black's turn (minimizing)
    :param alpha: Alpha value for pruning (best value for maximizing player)
    :param beta: Beta value for pruning (best value for minimizing player)
    :return: Best score and corresponding move
    """
    # Base case: If maximum depth is reached or the game is over, evaluate the position
    if depth == 0 or board.is_game_over():
        return evaluate_position(board), None

    legal_moves = list(board.legal_moves)
    best_move = None

    if maximizing_player:
        max_eval = float('-inf')
        for move in legal_moves:
            board.push(move)
            eval_score, _ = minimax(board, depth - 1, False, alpha, beta)  # Minimize for the opponent
            board.pop()
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:  # Beta cut-off
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for move in legal_moves:
            board.push(move)
            eval_score, _ = minimax(board, depth - 1, True, alpha, beta)  # Maximize for the opponent
            board.pop()
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:  # Alpha cut-off
                break
        return min_eval, best_move

# Example usage
board = chess.Board()

# Set depth for Minimax search
depth = 3

# Example usage
board = chess.Board("r1b2b1r/ppppkppp/8/3P4/5B2/2N5/PPP1B1PP/R4RK1 b - - 3 14")



# Find best move in the new position
score, best_move = minimax(board, depth, board.turn == chess.WHITE)
print(f"Best move: {board.san(best_move)} with score: {score}")


Best move: Kd8 with score: 195
